In [1]:
from bs4 import BeautifulSoup
import tensorflow as tf
import numpy as np

In [86]:
tf.expand_dims(tf.constant([[10,11], [12,13]]), axis=1)

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10, 11]],

       [[12, 13]]])>

In [89]:
l_in = tf.ragged.constant([[[1,2],[3,4]], [[5,6],[7,8],[9,10]]])
#l_in = tf.constant([[1,2],[3,4]])
print(l_in[:,1:])
print(tf.expand_dims([[0]], axis=0))
l_left_shift = tf.concat((l_in[:,1:], tf.constant([[[0]]])), axis=2)
print(l_left_shift)
mask_left_shift = tf.not_equal(l_in - l_left_shift, 0)
print(mask_left_shift)
mask = tf.concat(([True], mask_left_shift[:-1]), axis=1)
print(mask)
l_out = tf.boolean_mask(l_in, mask)

<tf.RaggedTensor [[[3, 4]], [[7, 8],
            [9, 10]]]>
tf.Tensor([[[0]]], shape=(1, 1, 1), dtype=int32)


InvalidArgumentError: {{function_node __wrapped__Equal_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [3] vs. [2] [Op:Equal]

In [2]:
data_file_path="C:/Users/aiden/Downloads/ProjectData/LGINKML/"
lg_file_path=data_file_path+"LGs/"
inkml_file_path=data_file_path+"INKMLs/"

In [4]:
def getSoup(file):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  #trace_strings = soup.find_all("trace")
  return soup
#def saveVocab():
# with open() as f:


In [5]:
def arrayToPoints (arr) :
  points = []
  for i in arr:
    #start=default_timer()
    #points.append(point(i[0],i[1]))
    points.append(point(float(i[0]), float(i[1])))
    #print(default_timer()-start)
  return(points)
#def raggedTensorToPoints (tensor) :
  
def PointsToArray (points) :
  arr=[]
  for point in points:
    arr.append([point.x,point.y])
  return arr

In [219]:
import numpy as np
from rdp import rdp
import math
EPSILON = 1e-12
MACHINE_EPSILON = 1.12e-16


class point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
#note calculations may be off because of int vs float
def isMachineZero (val) : 
  return (val >= -MACHINE_EPSILON and val <= MACHINE_EPSILON)
def hypot (x, y) : 
  return math.sqrt(x * x + y * y)
#def point (x, y) : return ( Point(x,y) )
def pointLength (p) : 
  return hypot(p.x, p.y)
def pointNegate (p) : 
  return point(-p.x, -p.y)
def pointAdd (p1, p2) : 
  return point(p1.x + p2.x, p1.y + p2.y)
def pointSubtract (p1, p2) : 
  return point(p1.x - p2.x, p1.y - p2.y)
def pointMultiplyScalar (p, n) : 
  return point(p.x * n, p.y * n)
def pointDot (p1, p2) : 
  return p1.x * p2.x + p1.y * p2.y
def pointDistance (p1, p2) : 
  return hypot(p1.x - p2.x, p1.y - p2.y)
def pointNormalize (p, length = 1) : 
  return pointMultiplyScalar(p, (length / pointLength(p)) if pointLength(p) else 0) #POSSIBLE FLAW
class createSegment: 
  def __init__(self, p, i, o):
    self.p=p
    self.i=i
    self.o=o #POSSIBLE FLAW (CHANGED)

def chordLengthParameterize (points, first, last)  :
    u = [0]
    for i in range(first+1, last+1):
    #for (let i = first + 1 i <= last i++) :
        #u[i - first] = u[i - first - 1] + pointDistance(points[i], points[i - 1])
        u.append(u[i - first - 1] + pointDistance(points[i], points[i - 1]))
    
    for i in range(1, last-first+1):
    #for (let i = 1, m = last - first i <= m i++) :
        u[i] /= u[last-first]
    
    return u

# Use Newton-Raphson iteration to find better root.
def findRoot (curve, point, u) :
    curve1 = []
    curve2 = []
    # Generate control vertices for Q'
    for i in range(3) : #FLAG
        curve1.append(pointMultiplyScalar(pointSubtract(curve[i + 1], curve[i]), 3))
    
    # Generate control vertices for Q''
    for i in range(2):
    #for (let i = 0 i <= 1 i++) :
        curve2.append(pointMultiplyScalar(pointSubtract(curve1[i + 1], curve1[i]), 2))
    
    # Compute Q(u), Q'(u) and Q''(u)
    pt = evaluate(3, curve, u)
    pt1 = evaluate(2, curve1, u)
    pt2 = evaluate(1, curve2, u)
    diff = pointSubtract(pt, point)
    df = pointDot(pt1, pt1) + pointDot(diff, pt2)
    # u = u - f(u) / f'(u)
    #return isMachineZero(df) ? u : u - pointDot(diff, pt1) / df
    return (u if isMachineZero(df) else (u-pointDot(diff,pt1)/df))

# Evaluate a bezier curve at a particular parameter value
def evaluate (degree, curve, t) :
    # Copy array
    tmp = curve[:]#.slice() FLAG
    # Triangle computation
    for i in range(1, degree+1):
    #for (let i = 1 i <= degree i++) :
        for j in range(degree):
        #for (let j = 0 j <= degree - i j++) :
            tmp[j] = pointAdd(pointMultiplyScalar(tmp[j], 1 - t), pointMultiplyScalar(tmp[j + 1], t))

    return tmp[0]

def addCurve (segments, curve)  :
    prev = segments[len(segments) - 1]
    prev.o = pointSubtract(curve[1], curve[0])
    segments.append(createSegment(curve[3], pointSubtract(curve[2], curve[3]), None))

# Use least-squares method to find Bezier control points for region.
def generateBezier (points, first, last, uPrime, tan1, tan2)  :
    epsilon = EPSILON
    #abs = Math.abs
    pt1 = points[first]
    pt2 = points[last]
    # Create the C and X matrices
    C = [[0, 0],[0, 0]]
    X = [0, 0]
    for i in range(0, last-first+1):
    #for (let i = 0, l = last - first + 1 i < l i++) :
        u = uPrime[i] 
        t = 1 - u
        b = 3 * u * t
        b0 = t * t * t
        b1 = b * t
        b2 = b * u
        b3 = u * u * u
        a1 = pointNormalize(tan1, b1)
        a2 = pointNormalize(tan2, b2)
        tmp = pointSubtract(pointSubtract(points[first + i], pointMultiplyScalar(pt1, b0 + b1)), pointMultiplyScalar(pt2, b2 + b3))
        C[0][0] += pointDot(a1, a1)
        C[0][1] += pointDot(a1, a2)
        # C[1][0] += a1.dot(a2)
        C[1][0] = C[0][1]
        C[1][1] += pointDot(a2, a2)
        X[0] += pointDot(a1, tmp)
        X[1] += pointDot(a2, tmp)
    
    # Compute the determinants of C and X
    detC0C1 = C[0][0] * C[1][1] - C[1][0] * C[0][1]
    alpha1=None
    alpha2=None
    if (abs(detC0C1) > epsilon) :
        # Kramer's rule
        detC0X = C[0][0] * X[1] - C[1][0] * X[0]
        detXC1 = X[0] * C[1][1] - X[1] * C[0][1]
        # Derive alpha values
        alpha1 = detXC1 / detC0C1
        alpha2 = detC0X / detC0C1
    
    else :
        # Matrix is under-determined, try assuming alpha1 == alpha2
        c0 = C[0][0] + C[0][1]
        c1 = C[1][0] + C[1][1]
        #alpha1 = alpha2 = abs(c0) > epsilon ? X[0] / c0 : abs(c1) > epsilon ? X[1] / c1 : 0
        alpha1 = alpha2 = X[0] / c0 if abs(c0) > epsilon else X[1] / c1 if abs(c1) > epsilon else 0
    
    # If alpha negative, use the Wu/Barsky heuristic (see text)
    # (if alpha is 0, you get coincident control points that lead to
    # divide by zero in any subsequent NewtonRaphsonRootFind() call.
    segLength = pointDistance(pt2, pt1)
    eps = epsilon * segLength
    handle1 = None
    handle2 = None
    if (alpha1 < eps or alpha2 < eps) :
        # fall back on standard (probably inaccurate) formula,
        # and subdivide further if needed.
        alpha1 = alpha2 = segLength / 3
    
    else :
        # Check if the found control points are in the right order when
        # projected onto the line through pt1 and pt2.
        line = pointSubtract(pt2, pt1)
        # Control points 1 and 2 are positioned an alpha distance out
        # on the tangent vectors, left and right, respectively
        handle1 = pointNormalize(tan1, alpha1)
        handle2 = pointNormalize(tan2, alpha2)
        if (pointDot(handle1, line) - pointDot(handle2, line) > segLength * segLength) :
            # Fall back to the Wu/Barsky heuristic above.
            alpha1 = alpha2 = segLength / 3
            handle1 = handle2 = None # Force recalculation
        
    
    # First and last control points of the Bezier curve are
    # positioned exactly at the first and last data points
    return [pt1, pointAdd(pt1, handle1 or pointNormalize(tan1, alpha1)), pointAdd(pt2, handle2 or pointNormalize(tan2, alpha2)), pt2]

# Given set of points and their parameterization, try to find
# a better parameterization.
def reparameterize (points, first, last, u, curve) :
    #for (let i = first i <= last i++) :
    for i in range(first, last+1):
        u[i - first] = findRoot(curve, points[i], u[i - first])
    
    # Detect if the new parameterization has reordered the points.
    # In that case, we would fit the points of the path in the wrong order.
    #for (let i = 1, l = u.length i < l i++) :
    for i in range (1, len(u)):
        if (u[i] <= u[i - 1]):
            return False
    
    return True


class error:
  def __init__(self, error, index):
    self.error = error
    self.index = index
# Find the maximum squared distance of digitized points to fitted curve.
def findMaxError (points, first, last, curve, u)  :
    index = math.floor((last - first + 1) / 2)
    maxDist = 0
    #for (let i = first + 1 i < last i++) :
    for i in range (first+1, last):
        P = evaluate(3, curve, u[i - first])
        v = pointSubtract(P, points[i])
        dist = v.x * v.x + v.y * v.y # squared
        if (dist >= maxDist) :
            maxDist = dist
            index = i
        
    return error(maxDist, index)
    #return :
    #   error: maxDist,
    #    index: index,
    



def fit (points, closed, error) :
    #points=filterRepeats(points)
    #points=rdpFilter(points,0.3)
    length = len(points)
    if length == 0 and type(length) == int:
        return []
    segments = [createSegment(points[0], None, None)]
    if length == 1:
      return segments
    fitCubic(points, segments, error, 0, length - 1, 
    pointSubtract(points[1], points[0]), 
    pointSubtract(points[length - 2], points[length - 1]))
    if (closed) :
        segments.pop(0)
        segments.pop()
    
    return segments
#note MAY HAVE TO FIND ALTERNATIVE TO === instead of using ==
def fitCubic (points, segments, error, first, last, tan1, tan2):
    #  Use heuristic if region only has two points in it
    if (last - first == 1) :
        pt1 = points[first]
        pt2 = points[last]
        dist = pointDistance(pt1, pt2) / 3
        addCurve(segments, [pt1, pointAdd(pt1, pointNormalize(tan1, dist)), pointAdd(pt2, pointNormalize(tan2, dist)), pt2])
        return
    
    # Parameterize points, and attempt to fit curve
    uPrime = chordLengthParameterize(points, first, last)
    maxError = max(error, error * error)
    split=None
    parametersInOrder = True
    # Try not 4 but 5 iterations
    #for (let i = 0 i <= 4 i++) :
    for i in range(5):
        curve = generateBezier(points, first, last, uPrime, tan1, tan2)
        #  Find max deviation of points to fitted curve
        Max = findMaxError(points, first, last, curve, uPrime)
        if (Max.error < error and parametersInOrder) :
            addCurve(segments, curve)
            #for segment in segments:
            #  print((str(segment.i.x) if segment.i else "None") + " " + (str(segment.i.y) if segment.i else "None") + " " + (str(segment.o.x) if segment.o else "None") + " " + (str(segment.o.y) if segment.o else "None") + " " + (str(segment.p.x) if segment.p else "None") + " " + (str(segment.p.y) if segment.p else "None"))
            return
        
        split = Max.index
        # If error not too large, try reparameterization and iteration
        if (Max.error >= maxError):
            break
        parametersInOrder = reparameterize(points, first, last, uPrime, curve)
        maxError = Max.error
    
    # Fitting failed -- split at max error point and fit recursively
    tanCenter = pointSubtract(points[split - 1], points[split + 1])
    fitCubic(points, segments, error, first, split, tan1, tanCenter)
    fitCubic(points, segments, error, split, last, pointNegate(tanCenter), tan2)


def filterRepeats_Normalize(points, min_x, min_y, max_y, precision_b=150):
  filtered_points=[]
  for i in range(len(points)):
    if i==len(points)-1 or points[i][0] != points[i+1][0] or points[i][1] != points[i+1][1]:
      filtered_points.append((np.array(points[i])-np.array([min_x, min_y]))*precision_b/(max_y-min_y if max_y-min_y != 0 else 1))
  return filtered_points


# Assign parameter values to digitized points
# using relative distances between points.

def svgPath (segments, closed, precision) :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      #return [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],[[segments[0].p.x,segments[0].p.y]]]
      return 'M' + str(segments[0].p.x) + "," + str(segments[0].p.y) #CHANGED
    
    if (not length):
        return ''
    for i in range(length):
        segment=segments[i]
        #repeat=False
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
        else :
            inX = curX + (segment.i.x if segment.i else 0)
            inY = curY + (segment.i.y if segment.i else 0)
            if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                parts.append('c' +
                      formatPair(outX - prevX, outY - prevY) +
                      ' ' +
                      formatPair(inX - prevX, inY - prevY) +
                      ' ' +
                      formatPair(curX - prevX, curY - prevY))

        prevX = curX
        prevY = curY
        outX = curX + (segment.o.x if segment.o else 0)
        outY = curY + (segment.o.y if segment.o else 0)
        first = False

    # Close path by drawing first segment again
    #return [allSegmentData, endPoints]
    return ''.join(parts)

def getSegmentsPathData (segments, closed, precision)  :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      return [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1]],[[segments[0].p.x,segments[0].p.y]]]
      #return 'M' + str(segments[0].p.x) + "," + str(segments[0].p.y) #CHANGED
    
    if (not length):
        return []
    allSegmentData=[]
    endPoints = []
    for i in range(length):
    #for (let i = 0 i < length i++)
        #addSegment(segments[i])
        #if not(prevX==segments[i].p.x and prevY==segments[i].p.y):
      segmentData = []
      segment=segments[i]
      #repeat=False
      curX = segment.p.x
      curY = segment.p.y
      if (first) :
          parts.append('M' + formatPair(curX, curY))
      else :
          inX = curX + (segment.i.x if segment.i else 0)
          inY = curY + (segment.i.y if segment.i else 0)
          if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                # c = relative curveto:
                #segmentData.extend([prevX, prevY, outX-prevX, outY-prevY, inX - prevX, inY - prevY, curX - prevX, curY - prevY])
              segmentData.extend([prevX, prevY, outX, outY, inX, inY, curX, curY])
              parts.append('c' +
                    formatPair(outX - prevX, outY - prevY) +
                    ' ' +
                    formatPair(inX - prevX, inY - prevY) +
                    ' ' +
                    formatPair(curX - prevX, curY - prevY))

              features = [segmentData[6]-segmentData[0], 
                          segmentData[7]-segmentData[1], 
                          math.sqrt((segmentData[0]-segmentData[2])**2+(segmentData[1]-segmentData[3])**2), 
                          math.sqrt((segmentData[6]-segmentData[4])**2+(segmentData[7]-segmentData[5])**2), 
                          math.atan2(segmentData[3]-segmentData[1], segmentData[2]-segmentData[0])-math.atan2(segmentData[7]-segmentData[1], segmentData[6]-segmentData[0]),
                          math.atan2(segmentData[5]-segmentData[7], segmentData[4]-segmentData[6])-math.atan2(segmentData[1]-segmentData[7], segmentData[0]-segmentData[6]), 
                          1]
              allSegmentData.append(features)
              endPoints.append([segmentData[6],segmentData[7]])

      prevX = curX
      prevY = curY
      outX = curX + (segment.o.x if segment.o else 0)
      outY = curY + (segment.o.y if segment.o else 0)
      first = False
    #print(allSegmentData)
    return [allSegmentData, endPoints]

class options:
  def __init__(self, closed, tolerance, precision):
    self.closed = closed
    self.tolerance = tolerance
    self.precision=precision

def f1(p):
  return point(p.x, p.y)

def f2(p):
  return point(float(p[0]), float(p[1]))

#def simplify(points, options) :
#  return fit(list(map((f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2), points)), options.closed, options.tolerance if options.tolerance else 2.5)
#def getFeatures (points, options):

def getSvgPath (points, options) :
    if (len(points) == 0 and type(len(points))==int) :
        return ''
    return svgPath(fit(list(map(f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)

def getFeatures (points, options)  :
    if (len(points) == 0 and type(len(points))==int) :
        return []
    return getSegmentsPathData(fit(list(map(f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)
    #return getSegmentsPathData(fit(list(map(f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)
    #return getSegmentsPathData(fit(points.map(typeof points[0].x == 'number' ? (p) => point(p.x, p.y) : (p) => point(p[0], p[1])), options.closed, options.tolerance ?? 2.5), options.closed, options.precision ?? 5)

In [257]:
error_files=[]
bezier_curve_errors=[]
parse_traces_errors=[]

In [344]:
from timeit import default_timer
def _parse_one_trace(trace_string):
  trace_split=trace_string.string
  if trace_split[0]=="\n":
    trace_split=trace_split[1:]
  if trace_split[-1]=="\n":
    trace_split=trace_split[:-1]
  trace_split=trace_split.split(",")
  a = [[float(s) for s in entry.strip().split(" ")[:2]] for entry in trace_split]
  return a


def getSvg(file, tolerance=15, precision_a=8, precision_b=150):
  #_, order = convert_relations(convert_inkml_to_lg(file))
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=""
  list_points=[]
  #order_dict=reorder(file.replace("INKMLs", "LGs").replace(".inkml", ".lg"))
  #for i in order:
  #  trace_string=trace_strings[i]
  #  list_points.append(_parse_one_trace(trace_string))
  for trace_string in trace_strings:
    list_points.append(_parse_one_trace(trace_string))
  array_points=tf.ragged.constant(list_points)
  mins=tf.math.reduce_min(array_points, axis=(0,1))
  maxs=tf.math.reduce_max(array_points, axis=(0,1))
  min_x=float(mins[0])
  min_y=float(mins[1])
  max_y=float(maxs[1])
  for points in list_points:
    points=filterRepeats_Normalize(points,min_x,min_y,max_y,precision_b)
    path=getSvgPath(points, options(False,tolerance,precision_a))
    a+=path
  return a 

def parse_traces(file, order, tolerance=15, precision_a=8, precision_b=150):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=[]
  first = True
  prev = None
  list_points=[]
  for i in order:
    try:
      trace_string=trace_strings[i]
    except:
      parse_traces_errors.append(trace_string)
    list_points.append(_parse_one_trace(trace_string))
  mins=tf.math.reduce_min(tf.ragged.constant(list_points), axis=(0,1))
  maxs=tf.math.reduce_max(tf.ragged.constant(list_points), axis=(0,1))
  min_x=float(mins[0])
  min_y=float(mins[1])
  max_y=float(maxs[1])
  for points in list_points:
    points=filterRepeats_Normalize(points,min_x,min_y,max_y,precision_b)
    #points=arrayToPoints(points)
    try:
      features=getFeatures(points, options(False,tolerance,precision_a))
    except:
      print("error bezier curve: ")
      bezier_curve_errors.append(file)
      print(file)
    if not first:
      try:
        dx=features[1][len(features[1])-1][0] - prev[1][len(prev[1])-1][0]
        dy=features[1][len(features[1])-1][1]-prev[1][len(prev[1])-1][1]
        #note if type error convert dx and dy to floats?
        a += [[dx, dy, math.sqrt(dx**2/9+dy**2/9), math.sqrt(dx**2/9+dy**2/9), 0.0, 0.0, 0]]
      except:
      #  print("path:")
      #  print(features)
      #  print("trace_string:")
      #  print(trace_string.string)
      #  print("file:")
      #  print(file)
        error_files.append(file)
      
    a += features[0]
    prev = features[:]
    first=False
  return a 

In [343]:
def convert_inkml_to_lg(file):
    return ("/".join(file.split("/")[:6])+"/LGs/"+"/".join(file.split("/")[7:])[:-5]+"lg")

In [153]:
#old y data generation

from collections import defaultdict
# todo: Add something to return the sequence of unique identifiers ('x_1' etc.)
def parse_symbols_and_alignments(file):
  with open(file, "r") as f:
    string = f.read()
  #lg=[]
  rel=0 #BOOLEAN sorta
  objects=[]
  rels=defaultdict(lambda: "NoRel")
  for part in string.split("\n"):
    line=part.split(", ")
    if (line[0]=="# Relations from SRT:"):
      rel=1
    else:
      if (line[0]!=''):
        if rel:
          #if rel > 1:
          rels[(line[1], line[2])]=line[3]
          #rel+=1
          #prev=line[1]
        else:
          if (line[0][0]=="#"):
            continue
          del line[3]
          objects.append((line[1:3], [int(l) for l in line[3:]]))
  def key(obj):
    return min(obj[1])
  objects=sorted(objects, key=key)
  output=[]

  output.append(objects[0][0][1])
  prev=objects[0][0][0]
  #print(rels)
  for ob in objects[1:]:
    #print((ob[0][0], prev))
    output.append(rels[(prev, ob[0][0])])
    output.append(ob[0][1])
    prev=ob[0][0]
    #output.append(ob[0][1])
    #output.append(rels[ob[0][0]])
  '''try:
    output.append(objects[-1][0][1])
  except:
    print(file)'''
  #return(objects, rels)
  return(output)



In [154]:
class Node:
  def __init__(self, symbol, relations, parent, strokes):
    #self.id = id
    self.symbol = symbol
    self.relations = relations
    self.parent = parent
    self.strokes = strokes
  '''def get_root(self, nodes):
    node=self
    while(nodes[node].parent!=None):
      node=nodes[node].parent
    #node=Node(self.symbol,self.relations,self.parent,self.strokes)
    #while(nodes[node.parent].parent != None):
    #  node=nodes[node.parent]
    return node'''

def get_root(node, nodes):
    while(nodes[node].parent!=None):
      node=nodes[node].parent
    return node


#nodes={"A_1" : Node("A_1", "A", ["Right"], ["x_1"], None, ), }

In [340]:
from collections import defaultdict
failures=0
def lg_to_tree(file):
  with open(file, "r") as f:
    string = f.read()
  rel=0 #BOOLEAN sorta
  nodes=defaultdict(lambda: Node(None, [], None, []))
  nodes_inside_relation=defaultdict(list)
  for part in string.split("\n"):
    line=part.split(", ")
    if (line[0]=="# Relations from SRT:"):
      rel=1
    else:
      if (line[0]!=''):
        if rel:
          #if rel > 1:
          if line[3] == "Inside":
            nodes_inside_relation[line[1]].append(line[2])
          nodes[line[1]].relations.append([line[2],line[3]])
          nodes[line[2]].parent=line[1]
          #rels[(line[1], line[2])]=line[3]
          #rel+=1
          #prev=line[1]
        else:
          if (line[0][0]=="#"):
            continue
          del line[3]
          nodes[line[1]].symbol=line[2]
          nodes[line[1]].strokes=[int(stroke) for stroke in line[3:]]
  for node in nodes_inside_relation:
    targets=nodes_inside_relation[node]
    if len(targets)>2:
      raise ValueError("more than 2 inside relation detected")
    if len(targets)==2:
      if targets[1] in [r[0] for r in nodes[targets[0]].relations]:
        for i, rel in enumerate(nodes[node].relations):
          if rel[0]==targets[1]:
            target_index=i
            break
        del nodes[node].relations[target_index]
      elif targets[0] in [r[0] for r in nodes[targets[1]].relations]:
        for i, rel in enumerate(nodes[node].relations):
          if rel[0]==targets[0]:
            target_index=i
            break
        del nodes[node].relations[target_index]
      else:
        print("-----FAILURE------")
        failures+=1
        return None

  
  return(nodes)



In [259]:
#length=0


def traverse_tree(tree, current, order, length):
  strokes=current.strokes
  relations_order={"Above":1, "Below":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  relations=sorted(current.relations, key = lambda rel: relations_order[rel[1]])
  def _add_current_strokes(strokes,length):
    strokes=sorted(strokes)
    for x in range(len(strokes)):
      order[strokes[x]]=length
      length+=1
    return length
  def _traverse_children(tree,relations,order,length):
    for i in range(len(relations)):
      order_length=traverse_tree(tree, tree[relations[i][0]], order, length)
      order=order_length[0]
      length=order_length[1]
    return length
  if len(relations)>0:
    if relations[0][1]=="Above":
      length = _traverse_children(tree,[relations[0]],order,length)
      length = _add_current_strokes(strokes,length)
      length = _traverse_children(tree,[relations[1]],order,length)
      #print("ABOVE",current.symbol, length)
      #length = _add_current_strokes(strokes,length)
    else:
      length = _add_current_strokes(strokes,length)
      #print("OTHER",current.symbol,length)
      length = _traverse_children(tree,relations,order,length)
  else:
    length = _add_current_strokes(strokes,length)
  #global length
  
  
  #if len(relations)==0:
  #print(current.symbol, length)
  return((order, length))

def reorder(file):
  tree=lg_to_tree(file)
  order = []
  root = get_root(list(tree.keys())[0], tree)
  #for i in range(len(root.strokes)):
    #order[root.strokes[i]]=i
  #global length
  length=0
  return(traverse_tree(tree, tree[root], order, length)[0])

In [157]:
def convert_sequence(sequence):
    symbols=sequence[::2]
    relations=sequence[1::2]
    return(symbols, relations)
#NOTe MAKE UTF NOT APPEAR FOR SUMS

In [325]:
#\frac{9x^2 + 5}{2}

#symbols = ['9','x','2','-','2','fractionbar','5','\\cdot ','x','2','fractionbar','5']
#relations = ["Right", "Sup", "DFS", "UTF", "NTB", "BTD", "UFD", "Right", "Sub", "OST", "BTD"]

symbols, relations=convert_sequence(['1', 'NTB', '-', 'BTD', 'r', 'Sup', '2', 'DFS-UFD', '=', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '-', 'Right', 'm', 'Right', ')', 'Sup', '2', 'DFS-UFD', '+', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '+', 'Right', 'm', 'Right', ')', 'Sup', '2'])

def parse_latex(symbols, relations):
  utf=0
  open_brackets=0
  #open_square_brackets=0
  latex_string=symbols[0]
  print(len(relations))
  print(len(symbols))
  print(relations)
  print(symbols)
  for i in range(len(relations)):
    s=symbols[i+1]
    #print(s)
    full_rel=relations[i]
    split_rel=full_rel.split('-')
    add_symbol=True
    #print(split_rel)
    for rel in split_rel:
      if rel in ["DFS", "UFS", "OFI", "UFD", "UFL", "DFL"]:
        #latex_string+=("}"+s)
        latex_string+="}"
        open_brackets-=1
      #if rel == "BTD":
        #latex_string+=(s+"}")
        #print(latex_string)
      if rel == "NTB":
        add_symbol=False
        #print(latex_string)
        if utf:
          latex_string+=("}{")
          utf-=1
        else:
          latex_string="\\frac{"+latex_string+"}{"
          open_brackets+=1
      if rel == "UTF":
        latex_string+=("\\frac{"+s)
        open_brackets+=1
        utf+=1
        add_symbol=False
      if rel in ["Sub", "LB", "DTI"]:
        latex_string+=("_{")
        open_brackets+=1
      if rel == "Sup":
        latex_string+=("^{")
        open_brackets+=1
      if rel == "Inside":
        latex_string+=("{")
        open_brackets+=1
      if rel == "STS":
        latex_string+=("}_{")
      if rel == "ITL":
        latex_string+=("}^{")
      if rel == "Radical":
        latex_string+="["
        #open_square_brackets+=1
      if rel == "RTI":
        latex_string+="]{"
        #open_square_brackets-=1
        open_brackets+=1
      #  latex_string+=s

      '''if rel == "OST":
        if utf:
          latex_string+=("}}{")
          utf-=1
        else:
          latex_string="\\frac{"+latex_string+"}}{"
          '''
      #if rel in ["Inside", "Right"]:
        #latex_string+=s
    if add_symbol:
      latex_string+=s
  latex_string+=("}")*open_brackets
  return latex_string
  
parse_latex(symbols, relations)

21
22
['NTB', 'BTD', 'Sup', 'DFS-UFD', 'UTF', 'NTB', 'BTD', 'Right', 'Right', 'Right', 'Right', 'Sup', 'DFS-UFD', 'UTF', 'NTB', 'BTD', 'Right', 'Right', 'Right', 'Right', 'Sup']
['1', '-', 'r', '2', '=', '1', '-', '(', 'R', '-', 'm', ')', '2', '+', '1', '-', '(', 'R', '+', 'm', ')', '2']
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


'\\frac{1}{r^{2}}=\\frac{1}{(R-m)^{2}}+\\frac{1}{(R+m)^{2}}'

In [278]:
missing_relations=set()
missing_relations_freq=defaultdict(lambda: 0)
pop_errors=[]

In [172]:
arr=[]

In [319]:
import pickle as pkl
from collections import defaultdict
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)
last_symbol=None
new_relations=[]
new_rels=defaultdict(lambda: [])
def traverse_relations(tree, current, new_rels, incoming_rel, prev_symbol, order):
  if tree[current].symbol=="\sum":
    relations_order={"Below":1, "Above":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  else:
    relations_order={"Above":1, "Below":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  
  #print(current)
  relations=sorted(tree[current].relations, key = lambda rel: relations_order[rel[1]])
  last_symbol=current
  if len(relations) > 0:
    no_rel=False
    #tree[current].symbol == "-" and 
    if relations[0][1]=="Above":
      if tree[current].symbol == "\int":
        relations[0][1]="Sup"
        if relations[1][1]=="Below":
          relations[1][1]="Sub"
        new_rels[current].append("Sup")
      elif tree[current].symbol == "\sum":
        new_rels[current].append("STL")
      elif tree[current].symbol == "\sqrt":
        new_rels[current].append("Radical")
      elif prev_symbol and len(new_rels[prev_symbol]):
        #if tree[prev_symbol].relations
        no_rel=True
        if new_rels[prev_symbol][-1]=="Right":
          new_rels[prev_symbol].pop()
        #print("prev",prev_symbol)
        new_rels[prev_symbol].append("UTF")
      else:
        no_rel=True
    elif relations[0][1]=="Below":
      if tree[current].symbol == "\sum":
        new_rels[current].append("DTI")
      else:
        new_rels[current].append("LB")
    else:
      new_rels[current].append(relations[0][1])
    if not no_rel and len(new_rels[current])==1:
      strokes=sorted(tree[current].strokes)
      #print(strokes)
      for x in strokes:
        order.append(x)
    for i in range(len(relations)):
      thing=traverse_relations(tree, relations[i][0], new_rels, relations[i][1], last_symbol, order)#CHANGED
      last_symbol=thing[0]
      new_rels=thing[1]
      #length=thing[2]
      order=thing[2]
      if relations[i][1]=="Above":
        if tree[current].symbol=="-":
          new_rels[current].append("BTD")
          if len(new_rels[current])==1:
            strokes=sorted(tree[current].strokes)
            for x in strokes:
              order.append(x)
              #length+=1
  else:
    new_rels[current]=[]
  add=True
  if incoming_rel=="Above":
    if tree[prev_symbol].symbol=="-": #CHANGE BECAUSE PREV SYMBOL IS THE LEAF OF SUM NOT SUM ITSELF
      #do stuff with sum
      new_rels[last_symbol].append("NTB")
    elif tree[prev_symbol].symbol=="\sqrt":
      new_rels[last_symbol].append("RTI")
    else:
      new_rels[last_symbol].append("DFL")
  elif incoming_rel=="Below":
    if tree[prev_symbol].symbol=="\lim":
      new_rels[last_symbol].append("UFL")
    elif tree[prev_symbol].symbol=="\sum":
      new_rels[last_symbol].append("ITL")
    else:
      new_rels[last_symbol].append("UFD")
  elif incoming_rel=="Sub":
    new_rels[last_symbol].append("UFS")
  elif incoming_rel=="Sup":
    if tree[prev_symbol].symbol=="\int":
      new_rels[last_symbol].append("STS")
    else:
      new_rels[last_symbol].append("DFS")
  elif incoming_rel=="Inside":
    new_rels[last_symbol].append("OFI")
  else:
    add=False
  if add:
    if len(new_rels[last_symbol])==1:
      strokes=sorted(tree[last_symbol].strokes)
      for x in strokes:
        order.append(x)
  return (last_symbol, new_rels, order)

def convert_relations(file):
  tree=lg_to_tree(file)
  root = get_root(list(tree.keys())[0], tree)
  order=[]
  try:
    traverse_output=traverse_relations(tree, root, defaultdict(lambda: []), None, None, order)
    new_rels=traverse_output[1]
    order=traverse_output[2]
    output=[]
    not_in_vocab=False
    for symbol in new_rels:
      #print(symbol)
      #print(new_rels[symbol])
      #arr.append(tree[symbol].symbol)
      output.append(vocab[tree[symbol].symbol])
      if len(new_rels[symbol]):
        relation="-".join(new_rels[symbol])
        if relation in vocab:
          output.append(vocab[relation])
          #arr.append(relation)
        else:
          print("missing: " +relation)
          missing_relations.add(relation)
          missing_relations_freq[relation]+=1
          not_in_vocab=True
          relation=relation.split("-")
          if relation.count("Right")>1 or relation.count("Inside")>1 or relation.count("Sup")>1 or relation.count("Sub")>1 or relation.count("BTD")>1:
            return (0, None)
  except:
    print("pop error: "+file)
    pop_errors.append(file)
    return (None, None)
  
  
  if not_in_vocab:
    return (None, None)
  #print(arr)
  return (output, order)

def get_latex(file):
  arr=[]
  tree=lg_to_tree(file)
  root = get_root(list(tree.keys())[0], tree)
  order=[]
  new_rels=defaultdict(lambda: [])
  new_rels=traverse_relations(tree, root, new_rels, None, None, order)[1]
  not_in_vocab=False
  for symbol in new_rels:
    if tree[symbol].symbol=="COMMA":
      arr.append(",")
    else:
      arr.append(tree[symbol].symbol)
    prev_rels=new_rels[symbol]
    if len(new_rels[symbol]):
      relation="-".join(new_rels[symbol])
      if relation=="DFS-Sub":
        relation="STS"
      if relation in vocab:
        arr.append(relation)
      else:
        print("missing: " +relation)
        missing_relations.add(relation)
        missing_relations_freq[relation]+=1
        not_in_vocab=True
  
  if not_in_vocab:
    return (None, None)
  
  if len(arr)%2==0:
    arr.pop()
  print(arr)
  converted_sequence=convert_sequence(arr)
  return parse_latex(converted_sequence[0], converted_sequence[1])


In [317]:
def add_to_vocab(symbol, vocab, index):
    for key in vocab:
        if vocab[key]>=index:
            vocab[key]+=1
    vocab[symbol]=index
    vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))
    #with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "wb") as f:
    #    pkl.dump(vocab, f)
def remove_from_vocab(vocab, index, symbol=None):
    for key in vocab:
        if vocab[key]>=index:
            vocab[key]-=1
    if symbol:
        vocab[symbol]=index
    vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))
    #with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "wb") as f:
    #    pkl.dump(vocab, f)

In [226]:
getSoup("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2014test/RIT_2014_25.inkml")

<?xml version="1.0" encoding="utf-8"?>
<ink xmlns="http://www.w3.org/2003/InkML">
<traceFormat>
<channel name="X" type="decimal"/>
<channel name="Y" type="decimal"/>
</traceFormat>
<annotation type="truth">$ \sin (t) / \cos (t) = \sin (t) / \cos (t) $</annotation>
<annotation type="writer">annot_225</annotation>
<annotation type="UI">2014_RIT_CROHME_25</annotation>
<annotation type="copyright">RIT_2014</annotation>
<annotationXML encoding="Content-MathML" type="truth">
<math xmlns="http://www.w3.org/1998/Math/MathML">
<mrow>
<mi xml:id="35:">\sin</mi>
<mrow>
<mo xml:id="37:">(</mo>
<mrow>
<mi xml:id="38:39:">t</mi>
<mrow>
<mo xml:id="40:">)</mo>
<mrow>
<mo xml:id="41:">/</mo>
<mrow>
<mi xml:id="42:43:">\cos</mi>
<mrow>
<mo xml:id="44:">(</mo>
<mrow>
<mi xml:id="45:46:">t</mi>
<mrow>
<mo xml:id="47:">)</mo>
<mrow>
<mo xml:id="48:49:">=</mo>
<mrow>
<mi xml:id="50:54:">\sin</mi>
<mrow>
<mo xml:id="55:">(</mo>
<mrow>
<mi xml:id="56:57:">t</mi>
<mrow>
<mo xml:id="58:">)</mo>
<mrow>
<mo xml:

In [346]:
#add_to_vocab("OFI-OFI", vocab, 25)
#vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))
#with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "wb") as f:
#    pkl.dump(vocab, f)

In [347]:
vocab

{'Right': 0,
 'Sub': 1,
 'Sup': 2,
 'Inside': 3,
 'UTF': 4,
 'NTB': 5,
 'BTD': 6,
 'UFD': 7,
 'DFS': 8,
 'UFS': 9,
 'OFI': 10,
 'DFS-NTB': 11,
 'LB': 12,
 'UFL': 13,
 'STS': 14,
 'DTI': 15,
 'ITL': 16,
 'DFL': 17,
 'UFS-NTB': 18,
 'DFS-UFD': 19,
 'UFS-UFD': 20,
 'DFS-DFS': 21,
 'OFI-NTB': 22,
 'OFI-UFD': 23,
 'DFS-OFI': 24,
 'OFI-OFI': 25,
 'UFD-DFS': 26,
 'Sup-UTF': 27,
 'UFS-UTF': 28,
 'UFL-UTF': 29,
 'Inside-UTF': 30,
 'UFD-OFI': 31,
 'DFS-OFI-UFD': 32,
 'NTB-UTF': 33,
 'UFS-UFS': 34,
 'DFL-UTF': 35,
 'UFD-UFD': 36,
 'UFS-OFI': 37,
 'UFS-DFS': 38,
 'Radical': 39,
 'RTI': 40,
 'RTI-UTF': 41,
 'Radical-UTF': 42,
 '!': 43,
 '(': 44,
 ')': 45,
 '+': 46,
 '-': 47,
 '.': 48,
 '/': 49,
 '0': 50,
 '1': 51,
 '2': 52,
 '3': 53,
 '4': 54,
 '5': 55,
 '6': 56,
 '7': 57,
 '8': 58,
 '9': 59,
 '<': 60,
 '=': 61,
 '>': 62,
 'A': 63,
 'B': 64,
 'C': 65,
 'COMMA': 66,
 'E': 67,
 'F': 68,
 'G': 69,
 'H': 70,
 'I': 71,
 'L': 72,
 'M': 73,
 'N': 74,
 'P': 75,
 'R': 76,
 'S': 77,
 'T': 78,
 'V': 79,
 'X':

In [ ]:
#vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))

In [281]:
bad_relation_files=[]

In [253]:
err=None

In [348]:
import pickle as pkl
import os
years=["2012", "2013", "2014train", "2014test", "2016", "2019"]
#years = ["2019"]
#train=False
#name="LG"
error_files=[]
bezier_curve_errors=[]
parse_traces_errors=[]
pop_errors=[]
bad_relation_files=[]
missing_relations=set()
missing_relations_freq=defaultdict(lambda: 0)
failures=0
#datapath="C:/Users/aiden/Downloads/ProjectData/"+year+"/"+ ("Train/" if train else "Test/") + name
X=[]
Y=[]
count=0
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)
for year in years:
    directory="C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/"+year+"/"
    for filename in os.listdir(directory):
        #the filename unincluding .inkml
        lgpath=convert_inkml_to_lg(directory+filename)
        #lgpath="C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/"+year+"/"+filename[:len(filename)-6]+".lg"
        if os.path.exists(lgpath):
            LG, order = convert_relations(lgpath)
            if order and LG:
                #try:
                X.append(parse_traces(directory+filename, order, tolerance=20))
                #except:
                #    print("parse_traces error")
                #    err=filename
                #    print(filename)
                Y.append(LG)
            else:
                if LG==0:
                    bad_relation_files.append(lgpath)
                #print("no corresponding relation in vocab: "+lgpath)

        else:
            print("missing file:")
            print(lgpath)
        count += 1

        if count % 100 == 0: print(count)
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
with open(datapath+"X_Y", "wb") as f:
    pkl.dump((X,Y), f)

100
missing: UFD-UTF
200
300
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
400
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
missing: UFD-STS
missing: UFD-DFS-UTF
missing: UFD-DFS-UFD
500
missing: DFS-UTF
600
missing: DFS-UTF
missing: DFS-UFD-OFI-UFD
missing: DFS-UTF
missing: DFS-UTF
missing: UFD-UTF
missing: UFD-DFS-NTB
700
missing: UFD-DFS-NTB
missing: UFS-UFD-OFI
missing: UFS-STS
800
missing: UFS-STS
missing: UFD-UTF
missing: DFS-UTF
900
missing: UFS-OFI-NTB
missing: UFD-NTB
missing: UFD-NTB
missing: DFS-DFS-DFS-DFS-DFS
1000
missing: UFS-ITL-UTF
missing: OFI-UFD-OFI
missing: OFI-UFD-OFI
1100
1200
1300
missing:

In [29]:
vocab

{'Right': 0,
 'Sub': 1,
 'Sup': 2,
 'Inside': 3,
 'UTF': 4,
 'NTB': 5,
 'BTD': 6,
 'UFD': 7,
 'DFS': 8,
 'UFS': 9,
 'OFI': 10,
 'DFS-NTB': 11,
 'LB': 12,
 'UFL': 13,
 'STS': 14,
 'DTI': 15,
 'ITL': 16,
 'DFL': 17,
 'UFS-NTB': 18,
 'DFS-UFD': 19,
 'UFS-UFD': 20,
 'DFS-DFS': 21,
 'OFI-NTB': 22,
 'OFI-UFD': 23,
 'DFS-OFI': 24,
 'UFD-DFS': 25,
 'Sup-UTF': 26,
 '!': 27,
 '(': 28,
 ')': 29,
 '+': 30,
 '-': 31,
 '.': 32,
 '/': 33,
 '0': 34,
 '1': 35,
 '2': 36,
 '3': 37,
 '4': 38,
 '5': 39,
 '6': 40,
 '7': 41,
 '8': 42,
 '9': 43,
 '<': 44,
 '=': 45,
 '>': 46,
 'A': 47,
 'B': 48,
 'C': 49,
 'COMMA': 50,
 'E': 51,
 'F': 52,
 'G': 53,
 'H': 54,
 'I': 55,
 'L': 56,
 'M': 57,
 'N': 58,
 'P': 59,
 'R': 60,
 'S': 61,
 'T': 62,
 'V': 63,
 'X': 64,
 'Y': 65,
 '[': 66,
 '\\Delta': 67,
 '\\alpha': 68,
 '\\beta': 69,
 '\\cos': 70,
 '\\div': 71,
 '\\exists': 72,
 '\\forall': 73,
 '\\gamma': 74,
 '\\geq': 75,
 '\\gt': 76,
 '\\in': 77,
 '\\infty': 78,
 '\\int': 79,
 '\\lambda': 80,
 '\\ldots': 81,
 '\\leq': 

In [283]:
#print(missing_relations)
print(dict(sorted(missing_relations_freq.items(), key=lambda item: item[1])))
#print(sorted(missing_relations_freq, key = lambda rel: missing_relations_freq[rel]))
'''print("pop_errors:")
print(pop_errors)
print("error_files")
print(error_files)
print("bezier_curve_errors")
print(bezier_curve_errors)
print("bad_relation_files:")
print(bad_relation_files)'''

{'DFS-DFS-DFS-DFS-DFS': 1, 'UFS-ITL-UTF': 1, 'DFS-UFD-UFD': 1, 'OFI-DFS-UFD': 1, 'UFD-ITL': 1, 'OFI-UTF': 1, 'DFS-UFD-UFD-OFI': 1, 'UFS-UFD-DFS': 1, 'DFS-UFS-UFD': 1, 'OFI-UFS': 1, 'OFI-UFD-UFD': 1, 'DFS-ITL': 1, 'UFD-DFL': 1, 'OFI-UFD-NTB': 1, 'OFI-OFI-OFI-OFI-OFI': 1, 'UFD-UFD-UFD-UFD-UFD': 1, 'UFD-OFI-DFL': 1, 'DFS-OFI-OFI-OFI': 1, 'UFS-UFS-DFS': 1, 'UFS-UFS-UFS': 1, 'DFS-OFI-UFD-UTF': 1, 'UFS-DFL-UTF': 1, 'DFS-UFS-OFI': 1, 'UFS-UFD-OFI': 2, 'UFS-OFI-NTB': 2, 'OFI-UFS-UTF': 2, 'STS-UTF': 2, 'UFD-OFI-UTF': 2, 'UFS-UFL-UTF': 2, 'UFS-UFD-UTF': 2, 'DFS-UFD-NTB': 2, 'OFI-UFD-UTF': 2, 'UFS-UFL': 3, 'DFS-UFD-UTF': 3, 'UFD-NTB-UTF': 3, 'OFI-OFI-UFD': 3, 'OFI-OFI-NTB': 3, 'UFS-ITL': 3, 'UFD-STS-UTF': 4, 'OFI-DFL': 4, 'UFS-OFI-UFD': 4, 'ITL-UTF': 5, 'OFI-UFD-OFI': 6, 'UFS-STS': 8, 'UFD-NTB': 8, 'UFD-UFS': 8, 'OFI-DFS': 9, 'UFD-DFS-UTF': 10, 'UFD-DFS-UFD': 10, 'UFD-STS': 10, 'DFS-UFS': 12, 'DFS-UFD-OFI-UFD': 13, 'DFS-DFS-DFS': 16, 'UFS-UFD-UFD-UFD': 16, 'UFD-UFD-UFD': 16, 'DFS-UTF': 17, 'OFI-N

'print("pop_errors:")\nprint(pop_errors)\nprint("error_files")\nprint(error_files)\nprint("bezier_curve_errors")\nprint(bezier_curve_errors)\nprint("bad_relation_files:")\nprint(bad_relation_files)'

In [285]:
get_latex()

[]


In [339]:
print(len(error_files))
print(len(pop_errors))
print(len(bad_relation_files))
print(bezier_curve_errors)
print(sum([missing_relations_freq[rel] for rel in missing_relations_freq.keys()]))

0
0
0
['C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2012/formulaire047-equation054.inkml']
695


In [338]:
#testing parse_traces
file='C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2012/formulaire047-equation054.inkml'
print(convert_inkml_to_lg(file))
parse_traces(file, convert_relations(convert_inkml_to_lg(file))[1])

C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/2012/formulaire047-equation054.lg
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


[[9.660784978414288,
  -6.108485320209141,
  6.635646573128413,
  7.3659752058556105,
  0.8080749186069507,
  -0.7361912978293754,
  1],
 [-32.03533027474153,
  19.83356748159254,
  8.804368957008686,
  31.499244448650888,
  -3.8872568213116008,
  -1.461098711017407,
  1],
 [21.865082412774548,
  -13.725082161383398,
  28.407049105554428,
  55.21869989289489,
  1.686688088736018,
  -2.6977764963259188,
  1],
 [-20.34049572393387,
  48.81718973744147,
  17.628429627409407,
  17.628429627409407,
  0.0,
  0.0,
  0],
 [-35.59650117718719,
  64.0757298319065,
  15.879640573907714,
  0.9078130531280216,
  -2.6643490819832314,
  -0.03529190941245641,
  1],
 [-2.0340495723933754,
  13.725082161383412,
  4.624995351345158,
  4.62499535134516,
  0.324664609287838,
  4.102859702378922,
  1],
 [119.50072920826305,
  4.068099144786757,
  39.85665107183552,
  39.85665107183552,
  0.0,
  0.0,
  0],
 [-23.900399305773732,
  20.86009717233345,
  18.61941670454667,
  0.9211045694064938,
  -3.67231006017

In [287]:
vocab

{'Right': 0,
 'Sub': 1,
 'Sup': 2,
 'Inside': 3,
 'UTF': 4,
 'NTB': 5,
 'BTD': 6,
 'UFD': 7,
 'DFS': 8,
 'UFS': 9,
 'OFI': 10,
 'DFS-NTB': 11,
 'LB': 12,
 'UFL': 13,
 'STS': 14,
 'DTI': 15,
 'ITL': 16,
 'DFL': 17,
 'UFS-NTB': 18,
 'DFS-UFD': 19,
 'UFS-UFD': 20,
 'DFS-DFS': 21,
 'OFI-NTB': 22,
 'OFI-UFD': 23,
 'DFS-OFI': 24,
 'UFD-DFS': 25,
 'Sup-UTF': 26,
 'UFS-UTF': 27,
 'UFL-UTF': 28,
 'Inside-UTF': 29,
 'UFD-OFI': 30,
 'DFS-OFI-UFD': 31,
 'NTB-UTF': 32,
 'UFS-UFS': 33,
 'DFL-UTF': 34,
 'UFD-UFD': 35,
 'UFS-OFI': 36,
 'UFS-DFS': 37,
 '!': 38,
 '(': 39,
 ')': 40,
 '+': 41,
 '-': 42,
 '.': 43,
 '/': 44,
 '0': 45,
 '1': 46,
 '2': 47,
 '3': 48,
 '4': 49,
 '5': 50,
 '6': 51,
 '7': 52,
 '8': 53,
 '9': 54,
 '<': 55,
 '=': 56,
 '>': 57,
 'A': 58,
 'B': 59,
 'C': 60,
 'COMMA': 61,
 'E': 62,
 'F': 63,
 'G': 64,
 'H': 65,
 'I': 66,
 'L': 67,
 'M': 68,
 'N': 69,
 'P': 70,
 'R': 71,
 'S': 72,
 'T': 73,
 'V': 74,
 'X': 75,
 'Y': 76,
 '[': 77,
 '\\Delta': 78,
 '\\alpha': 79,
 '\\beta': 80,
 '\\cos'

In [326]:
get_latex("C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/2014train/109_caue.lg")

['\\sqrt', 'Radical', 'x', 'RTI-UTF', 'a', 'NTB', '-', 'BTD', 'b', 'UFD-OFI', '=', 'UTF', '\\sqrt', 'Radical', 'x', 'RTI', 'a', 'OFI-NTB', '-', 'BTD', '\\sqrt', 'Radical', 'x', 'RTI', 'b']
12
13
['Radical', 'RTI-UTF', 'NTB', 'BTD', 'UFD-OFI', 'UTF', 'Radical', 'RTI', 'OFI-NTB', 'BTD', 'Radical', 'RTI']
['\\sqrt', 'x', 'a', '-', 'b', '=', '\\sqrt', 'x', 'a', '-', '\\sqrt', 'x', 'b']
1
2
3
4
5
6
7
8
9
10
11
12


'\\sqrt[x]{\\frac{a}{b}}=\\frac{\\sqrt[x]{a}}{\\sqrt[x]{b}}'

In [ ]:
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab=pkl.load(f)

In [142]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
with open(datapath+"X_Y", "rb") as f:
    X, Y = pkl.load(f)

In [ ]:
length = len(X)
vocab_size = len(vocab)
X_train = []
Y_train = []
X_test = []
Y_test = []
for i in range(length):
    if (i%10):
        X_train.append(X[i])
        Y_train.append(Y[i])
    else:
        X_test.append(X[i])
        Y_test.append(Y[i])


In [149]:
X[13]

[[53.976357587037185,
  -6.8203097444636285,
  0.04232996102454726,
  9.527255840937514,
  -1.8994071709512277,
  0.12569124253668873,
  1],
 [-3.969633405956799,
  0.0,
  1.3232111353189282,
  1.3232111353189282,
  -3.141592653589793,
  0.0,
  1],
 [-0.5727994511950243,
  1.1322779849214797,
  0.8955113932292278,
  0.7601925204564282,
  1.1024539166069802,
  -0.4683424101879161,
  1],
 [-5.674710842072706,
  50.566202714805655,
  10.624515590523309,
  9.75200728492433,
  -0.11175580413755393,
  0.35658660605077963,
  1],
 [-41.481337000501725,
  4.542432857151525,
  10.520534040429869,
  11.282598585565776,
  -0.9933830444386507,
  0.23881698800514872,
  1],
 [-18.1830523460773,
  -3.396833954762073,
  6.165872683447995,
  6.165872683447996,
  -0.054938264249239754,
  -2.857934623488246,
  1],
 [27.28123897785097,
  -27.840717511576543,
  12.993038955644328,
  12.993038955644328,
  0.0,
  0.0,
  0],
 [-36.925583225880175,
  -6.247510293267723,
  9.8703534580207,
  9.299204646900659,
 

In [ ]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"

In [ ]:
with open(datapath+"Train", "wb") as f:
    pkl.dump((X_train, Y_train, vocab), f)
with open(datapath+"Test", "wb") as f:
    pkl.dump((X_test, Y_test, vocab), f)

In [ ]:
X_train=[]
Y_train=[]

In [72]:
with open(datapath+"Train", "rb") as f:
    X_train, Y_train = pkl.load(f)
with open(datapath+"Test", "rb") as f:
    X_test, Y_test = pkl.load(f)

In [ ]:
print(len(X_train[5]))

37


In [78]:
NUM_FEATURES=7
len_train=len(X_train)
len_test=len(X_test)
max_X_train_len = max([len(sample) for sample in X_train])
max_Y_train_len = max([len(sample) for sample in Y_train])
max_X_test_len = max([len(sample) for sample in X_test])
max_Y_test_len = max([len(sample) for sample in Y_test])
padding_vector=np.ones(NUM_FEATURES)
eos_index=130

In [79]:
X_train_data_len=[len(data) for data in X_train]
Y_train_data_len=[len(data) for data in Y_train]
X_test_data_len=[len(data) for data in X_test]
Y_test_data_len=[len(data) for data in Y_test]

In [80]:
padded_X_train=np.zeros((len_train, max_X_train_len, NUM_FEATURES))+padding_vector
padded_Y_train=np.zeros((len_train, max_Y_train_len))+eos_index
padded_X_test=np.zeros((len_test, max_X_test_len, NUM_FEATURES))+padding_vector
padded_Y_test=np.zeros((len_test, max_Y_test_len))+eos_index

In [97]:
for i in range(len_train):
    try:
        sample_X = np.array(X_train[i])
        padded_X_train[i,:len(sample_X),:]=sample_X
        sample_Y = np.array(Y_train[i])
        padded_Y_train[i,:len(sample_Y)]=sample_Y
    except:
        
for i in range(len_test):
    sample_X = np.array(X_test[i])
    padded_X_test[i,:len(sample_X),:]=sample_X
    sample_Y = np.array(Y_test[i])
    padded_Y_test[i,:len(sample_Y)]=sample_Y


(91, 7)
(91, 7)
(21, 7)
(21, 7)
(61, 7)
(61, 7)
(11, 7)
(11, 7)
(29, 7)
(29, 7)
(77, 7)
(77, 7)
(92, 7)
(92, 7)
(27, 7)
(27, 7)
(31, 7)
(31, 7)
(18, 7)
(18, 7)
(36, 7)
(36, 7)
(63, 7)
(63, 7)
(11, 7)
(11, 7)
(22, 7)
(22, 7)
(56, 7)
(56, 7)
(26, 7)
(26, 7)
(51, 7)
(51, 7)
(135, 7)
(135, 7)
(32, 7)
(32, 7)
(18, 7)
(18, 7)
(47, 7)
(47, 7)
(67, 7)
(67, 7)
(79, 7)
(79, 7)
(76, 7)
(76, 7)
(71, 7)
(71, 7)
(48, 7)
(48, 7)
(17, 7)
(17, 7)
(58, 7)
(58, 7)
(55, 7)
(55, 7)
(59, 7)
(59, 7)
(59, 7)
(59, 7)
(63, 7)
(63, 7)
(115, 7)
(115, 7)
(76, 7)
(76, 7)
(82, 7)
(82, 7)
(52, 7)
(52, 7)
(59, 7)
(59, 7)
(72, 7)
(72, 7)
(69, 7)
(69, 7)
(103, 7)
(103, 7)
(83, 7)
(83, 7)
(92, 7)
(92, 7)
(49, 7)
(49, 7)
(46, 7)
(46, 7)
(58, 7)
(58, 7)
(63, 7)
(63, 7)
(96, 7)
(96, 7)
(74, 7)
(74, 7)
(86, 7)
(86, 7)
(56, 7)
(56, 7)
(60, 7)
(60, 7)
(60, 7)
(60, 7)
(69, 7)
(69, 7)
(64, 7)
(64, 7)
(83, 7)
(83, 7)
(94, 7)
(94, 7)
(58, 7)
(58, 7)
(57, 7)
(57, 7)
(66, 7)
(66, 7)
(68, 7)
(68, 7)
(74, 7)
(74, 7)
(106, 7)
(106, 7)


AssertionError: [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 7, 6, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

In [99]:
print(len(Y_train))
print(len(X_test))

8971
997
